In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import gc

# 1. Gerar os dados de treino
np.random.seed(42)
torch.manual_seed(42)

num_samples = 100
angles_train_np = np.random.uniform(0, 4 * np.pi, num_samples).reshape(-1, 1)
sin_values_train_np = np.sin(angles_train_np)
noise = np.random.normal(0, 0.1, sin_values_train_np.shape)
sin_values_train_np += noise

# Converter os dados para tensores PyTorch
angles_train = torch.tensor(angles_train_np, dtype=torch.float32)
sin_values_train = torch.tensor(sin_values_train_np, dtype=torch.float32)

# 2. Definir a rede neural totalmente conectada (FCNN)
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 15),
            nn.Tanh(),
            nn.Linear(15, 15),
            nn.Tanh(),
            nn.Linear(15, 15),
            nn.Tanh(),
            nn.Linear(15, 1)
        )

    def forward(self, x):
        return self.net(x)

# Instanciar o modelo
model = FCNN()

# Definir a função de perda e o otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 3. Treinar o modelo
num_epochs = 1000  # Reduced from 10000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(angles_train)
    loss = criterion(outputs, sin_values_train)
    loss.backward()
    optimizer.step()

    # (Opcional) imprimir a perda a cada 200 iterações
    if (epoch+1) % 200 == 0:
        print(f'Época [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

    # Clear unnecessary variables to avoid memory issues
    del outputs, loss
    gc.collect()

# 4. Gerar dados de teste
num_test_samples = 100
angles_test_np = np.linspace(0, 6 * np.pi, num_test_samples).reshape(-1, 1)
sin_values_true_np = np.sin(angles_test_np)
angles_test = torch.tensor(angles_test_np, dtype=torch.float32)

# 5. Fazer previsões
model.eval()
with torch.no_grad():
    sin_values_predicted = model(angles_test).detach().numpy()

# 6. Avaliar o modelo
mse = np.mean((sin_values_true_np - sin_values_predicted)**2)
print(f"Mean Squared Error on Test Data: {mse:.6f}")

# 7. Visualizar os resultados
plt.figure(figsize=(10, 6))
plt.scatter(angles_train_np, sin_values_train_np, label='Training Data', alpha=0.5)
plt.plot(angles_test_np, sin_values_true_np, label='True sin(theta)', color='blue')
plt.plot(angles_test_np, sin_values_predicted, label='Predicted sin(theta)', color='red')
plt.xlabel('Angle (radians)')
plt.ylabel('sin(theta)')
plt.title('FCNN Interpolation of sin(theta) with PyTorch')
plt.legend()
plt.grid(True)
plt.show()


Época [200/1000], Loss: 0.096001
Época [400/1000], Loss: 0.046311
Época [600/1000], Loss: 0.010954
Época [800/1000], Loss: 0.007939
Época [1000/1000], Loss: 0.007672
Mean Squared Error on Test Data: 0.500544


: 